# 1. Word2Vec 모델 학습을 위한 전처리


## 1) Tag 데이터 로드

먼저 태그데이터를 읽어옵니다. <br>
pandas를 사용하여 읽어온 데이터를 데이터프레임 형태로 변환합니다. <br>
데이터는 MovieLens의 데이터셋을 사용하였습니다. <br>


In [1]:
import pandas as pd

tag = pd.read_csv('./data/tags_large.csv')
tag_df = pd.DataFrame(tag)
tag_df.head(10)
# 유저별 영화별 태그 데이터입니다.


,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195
5,14,318,justice,1442615192
6,14,480,Dinosaurs,1443148563
7,14,593,psychothriller,1444014286
8,14,1682,philosophy,1442615158
9,14,1682,surveillance,1442615167


## 2) 유저별 태그데이터 배열 생성
로드한 데이터를 잘라서 유저별 영화별 배열을 생성합니다. <br>

<br>

#### 예 : <br>
#### [유저1-영화1-태그1  ,  유저1-영화1-태그2  ,  유저1-영화1-태그3 , ...] <br>
가장 먼저 태그들이 정렬됩니다. 이때, 태그는 시간의 순서에 따라 정렬합니다. <br>
비슷한시간대에 비슷한 태그를 작성한다는 가정하에 정렬하였습니다. <br>

<br>

#### [유저1-영화2-태그1  ,  유저1-영화2-태그2  ,  유저1-영화2-태그3 , ...] <br>
영화가 바뀌면 새로운 어레이가 생성됩니다. <br>
유저1이 영화1번, 영화2번에 작성한 태그는 각각 다른 어레이에 들어갑니다. <br>

<br>

#### [유저2-영화1-태그1  ,  유저2-영화1-태그2  ,  유저2-영화1-태그3 , ...] <br>
유저 1이 작성한 태그가 끝나면 유저2의 태그가 시작됩니다. <br>
마찬가지로 같은영화에 작성한 태그는 같은 어레이에 들어가고 <br>
다른영화에 작성한 태그는 다른 어레이에 들어갑니다. <br>
유저2가 끝나면 유저n 까지 반복합니다. <br> 


In [2]:
from src.word2vec_mylib import tag_manager as tm


tag_array = tm.get_tag_by_user(tag_df, start_movie=110, start_user=14)

for i in range(0, 10):
    print(tag_array[i])
    
# 위의 방식대로 처리한 태그데이터가 담긴 배열입니다.

['medieval', 'epic']
['sci-fi', 'space action']
['justice', 'imdb top 250']
['dinosaurs']
['psychothriller']
['philosophy', 'surveillance']
['epic']
['pixar']
['dinosaurs']
['sci-fi', 'classic sci-fi', 'harrison ford', 'must see']


# 2. Word2Vec 모델 학습


## 1) 모델 학습
<br>
위처럼 생성한 배열을 Word2Vec 모델에 넣어 학습시킵니다. <br>
Word2Vec 모델은 Gensim의 라이브러리를 사용하였습니다. <br> 
<br>
Skip gram 방식을 사용하였고, Window Size는 256 차원입니다. <br>
100번 이하로 나온 태그들은 제거하고, 200번 학습을 시켰습니다. <br>


In [73]:
from gensim.models.word2vec import Word2Vec

model = Word2Vec(tag_array, 
                 size=256, 
                 window=3, 
                 min_count=100, 
                 workers=8, 
                 iter=100, 
                 sg=1)

model.init_sims(replace=True)

C:\Users\ASUS\Desktop\Github\Movie-Recommender\venv\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## 2) 모델 테스트

<br>

모델 학습이 잘 되었는지 테스트를 해봅니다. <br>
marvel 태그와 비슷한 태그를 출력합니다. <br>

In [75]:
print("marvel 태그와 유사한 태그 목록 :  \n")
model.wv.most_similar("marvel")


marvel 태그와 유사한 태그 목록 :  



C:\Users\ASUS\Desktop\Github\Movie-Recommender\venv\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('marvel cinematic universe', 0.5597065687179565),
 ('mcu', 0.5485382080078125),
 ('the avengers', 0.5097784399986267),
 ('superhero', 0.5034318566322327),
 ('comic book', 0.49842318892478943),
 ('thor', 0.4981544017791748),
 ('iron man', 0.4667408764362335),
 ('x-men', 0.455301433801651),
 ('captain america', 0.45062339305877686),
 ('superheroes', 0.40893399715423584)]

In [ ]:
from src.word2vec_mylib import tag_manager as tm
from src.word2vec_mylib import movie_manager as mm

movie = pd.read_csv('./data/movies_large.csv')
movie_df = pd.DataFrame(movie)

drop_tag_df = mm.get_movieId_tag_df(tag_df)

numberOfAll = 0
testMovieId = 5456

TF_list = tm.get_semantic_tag(testMovieId, drop_tag_df)

print(mm.id_to_name(testMovieId, movie_df), "\n")
print(TF_list, "\n")

for tag, numberOfTimes in TF_list:
    numberOfAll += numberOfTimes

result = []
for tag, numberOfTimes in TF_list:
    weightForTag = (numberOfTimes / numberOfAll) * 10
    index = round(weightForTag)
    result.append(tag)  # 그 영화의 유효태그 3개를 담는 과정
    for similar in model.wv.most_similar(tag):
        result.append(similar[0])  # 모델에서 유사한 태그들을 가중치에 맞게 담는 과정
        index = index - 1
        if index == 0:
            break

disinct_list = list(set(result))
print(disinct_list)

print("\n")

      movieId             title          genres
5358     5454  Mo' Money (1992)  Comedy|Romance 

[['con artist', 1], ['credit card', 1], ['swindler', 1]] 



KeyError: "word 'con artist' not in vocabulary"



# 4. 평가섹션을 위한 전처리




## 1) 평점데이터 로드 

<br>

TF-IDF 모델을 적용하기위해 평점 데이터를 로드합니다. <br>
평점 데이터는 MovieLens의 데이터셋을 사용하였습니다. <br>



In [102]:
rating = pd.read_csv('./data/ratings_large.csv')
rating_df = pd.DataFrame(rating)
rating_df.head(10)

# 평점 데이터입니다.

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264
5,1,1590,2.5,1256677236
6,1,1591,1.5,1256677475
7,1,2134,4.5,1256677464
8,1,2478,4.0,1256677239
9,1,2840,3.0,1256677500


## 2) 평균평점 3점 아래 영화 제거 
<br>

3점 아래의 영화는 추천해줄만한 영화가 아니라고 판단하고 <br>
추천 영화목록에서 제거합니다.<br>

In [106]:
from src.word2vec_mylib import movie_manager as mm

over_three_df = mm.get_over_three_movie(rating_df)
over_three_df.head(10)

# 평균평점 3점 이상의 영화 목록입니다.

,movieId
0,1
1,2
2,3
4,5
5,6
6,7
7,8
8,9
9,10
10,11


# 5. 평가 섹션

## 1) 영화 데이터 로드
<br>
우리가 만든 모델이 잘 동작하는지 확인하기 위해 <br>
영화 데이터를 로드합니다. <br>
마찬가지로 MovieLens의 데이터를 사용하였습니다

In [ ]:
movie = pd.read_csv('./data/movies_large.csv')
movie_df = pd.DataFrame(movie)
movie_df.head(10)

# 영화 이름과 장르 데이터입니다. 

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,u (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller
